# Construcción aplicada de un sistema de Query Answering


En este notebook vamos a construir un sistema de Query Answering usando la técnica RAG para que responda preguntas sobre el libro `El principito`. Para ello, primero vamos a construir un dataset a partir del libro y, posteriormente, construir el sistema usando las clases vistas en los notebooks anteriores.

In [ ]:
from huggingface_hub import login
token = ""
print("Hugging Face logging")
login(token)

In [ ]:
import torch
device_setup = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
print("Using: ", device_setup)

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"




#### Tarea QAC1 - Dataset
Lo primero que necesitamos hacer es preparar la base documental con la que alimentaremos nuestro RAG. Para ello, vamos a leer el PDF del libro y vamos a dividirlo en documentos relevantes que pueden ser frases, parrafos, etc. Para leer el fichero pdf podemos usar la libreria `pdfplumber`. Por otro lado, el libro se encuentra en la carpeta `./provided/el_principito.pdf`

In [ ]:
import pdfplumber


# TODO: Leer el fichero PDF y partirlo en documentos relevantes

documents = []
print("Tamaño del corpus documental: ",len(documents))


### Preguntas y respuestas para la evaluación

A continuación, vamos a utilizar las siguientes preguntas y respuestas para evaluar el modelo. Las preguntas han sido escogidas aleatoriamente de distintos exámenes tipo test online sobre el libro.

In [ ]:

questions = {
	1 : "¿El Principito vive en un asteroide?",
	2 : "¿El asteroide se llama B-612?",
	3 : "¿El Principito tiene una flor que cuida mucho?",
	4 : "¿La flor del Principito es una rosa?",
	5 : "¿El Principito se encuentra con un zorro en la Tierra?",
	6 : "¿El zorro le enseña al Principito sobre la amistad?",
	7 : "¿El aviador arregla su avión durante la historia?",
	8 : "¿El Principito considera los baobabs una amenaza para su asteroide?",
	9 : "¿El Principito piensa que los adultos siempre entienden las cosas importantes?",
	10 : "¿El Principito regresa a su asteroide al final del libro?"
}

responses = {
    1 : "Si.",
    2 : "Si.",
    3 : "Si.",
    4 : "Si.",
    5 : "Si.",
    6 : "Si.",
    7 : "Si.",
    8 : "Si.",
    9 : "No.",
    10 : "Implícitamente, si."
}

## Evaluar los modelos usando el dataset y el libro

In [ ]:
from libs.qa_lib import QuestionAnsweringFactory
from libs.qa_lib import LLMModel
from libs.qa_lib import evaluate_qa_answers


#### Tarea QAC2.1

Escribir el código necesario para que un modelo LLM basado en `TinyLlama/TinyLlama-1.1B-Chat-v1.0` responsa cada pregunta del dataset. Imprima por pantalla la pregunta, la respuesta esperada, y la generada

In [ ]:
answers = {}

for idx, question in questions.items():
    pass
    # TODO: Crear un LLModel sin contexto
    # TODO: dada una pregunta generar la respuesta
    # TODO: imprimir la pregunta, la respuesta esperada y la obtenida
    # TODO: guardar en la variable answers el nombre del modelo, el id de la pregunta y la respuesta obtenida

#### Tarea QAC2.2

Escribir el código necesario para que un modelo de Question Answering basado en `TinyLlama/TinyLlama-1.1B-Chat-v1.0` responsa cada pregunta del dataset. Imprima por pantalla la pregunta, la respuesta esperada, y la generada. Pruebe a variar la temperatura y observar las distintas respuestas.

In [ ]:
# TODO: construir un objeto de la clase QuestionAnswering

In [ ]:
# TODO: variar el top_k y la temperatura para mostrar como pueden mejorar las respuestas
for idx, question in questions.items():
    pass
    # TODO: dada una pregunta generar la respuesta
    # TODO: imprimir la pregunta, la respuesta esperada y la obtenida
    # TODO: guardar en la variable answers el nombre del modelo, el id de la pregunta y la respuesta obtenida

#### Tarea QAC3

Evalúe los dos modelos anteriores para ello utilice las funciones vistas en los notebooks anteriores

In [ ]:
from libs.rag_lib import plot_evaluation_results
# TODO: usando la funcion `evaluate_qa_answers` y `plot_evaluation_results` evaluar los resultados